In [45]:
import face_recognition
import os
from PIL import Image as ImagePIL
import numpy as np
import pandas as pd
import cv2
from cvzone.PoseModule import PoseDetector

In [46]:
def mascaraFrame(numFrame,video_path):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video file opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        exit()
    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, numFrame)

    # Read the frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if ret:
        # Display the frame
        cap.release()
        return frame
    else:
        print(f"Error: Could not read frame {numFrame}")

    # Release the video capture object and close all OpenCV windows
    cap.release()


In [47]:
def procesarVideo(video_path):
    # Initialize the PoseDetector
    detector = PoseDetector()
    respond = []
    maxFrames = 63
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video file opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        exit()

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Initialize variables to store the bounding box dimensions
    rect_x, rect_y, rect_w, rect_h = 0, 0, 0, 0
    Numframe = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if Numframe > maxFrames:
            return respond
            

        # Find the pose in the frame
        img = detector.findPose(frame, draw=False)
        
        # Get the list of landmarks
        lmList, _ = detector.findPosition(img, bboxWithHands=False)
        
        if lmList:
            # Calculate bounding box dimensions based on landmarks
            x_min = min(lmList, key=lambda x: x[0])[0]
            x_max = max(lmList, key=lambda x: x[0])[0]
            y_min = min(lmList, key=lambda x: x[1])[1]
            y_max = max(lmList, key=lambda x: x[1])[1]
            
            # Update rectangle dimensions
            rect_x = x_min
            rect_y = y_min
            rect_w = x_max - x_min
            rect_h = y_max - y_min
        
        # Ensure rectangle is within frame boundaries
        rect_x = max(0, rect_x)
        rect_y = max(0, rect_y)
        rect_w = min(width - rect_x, rect_w)
        rect_h = min(height - rect_y, rect_h)
        
        if rect_w > 0 and rect_h > 0:
            # Draw the rectangle on the frame
            cv2.rectangle(frame, (rect_x, rect_y), (rect_x + rect_w, rect_y + rect_h), (255, 0, 0), 2)
            
            # Crop the frame to the rectangle region
            
            maskFrame = mascaraFrame(Numframe,video_path)
            cropped_frame = maskFrame[rect_y:rect_y + rect_h, rect_x:rect_x + rect_w]

            if cropped_frame.size != 0:
                # Resize cropped frame back to original frame size
                resized_cropped_frame = cv2.resize(cropped_frame, (150, 150))
                respond.append(resized_cropped_frame)
                
        # Display the frame with the pose and bounding box (for debugging purposes)
        #cv2.imshow('Frame', frame)
        Numframe += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and writer objects and close all OpenCV windows
    cap.release()
    return respond

In [48]:
from database import Conexiones as con
def getKeypoints(etiqueta):
    cur, connectio = con.conexion()
    cur.execute("select nose,left_shoulder,right_shoulder,left_elbow,right_elbow,left_wrist,right_wrist,left_hip,right_hip,left_knee,right_knee,left_ankle,right_ankle,mid_shoulder,mid_hip from fun_generar_reporte_keypoints('"+etiqueta+"') where CAST(frame as integer) < 63")
    keypoints = cur.fetchall()
    return keypoints
    

In [49]:
def getAngles(etiqueta):
    cur, connectio = con.conexion()
    cur.execute("select nose_mid_shoulder,mid_shoulder_mid_hip,left_shoulder_left_elbow,left_elbow_left_wrist,right_shoulder_right_elbow,right_elbow_right_wrist,left_hip_left_knee,left_knee_left_ankle,right_hip_right_knee,right_knee_right_ankle,mid_shoulder_angle,left_shoulder_angle,left_elbow_angle,right_shoulder_angle,right_elbow_angle,left_hip_angle,left_knee_angle,right_hip_angle,right_knee_angle from fun_generar_reporte_angles('"+etiqueta+"') where CAST(frame as integer) < 63")
    angles = cur.fetchall()
    return angles

In [56]:

videos = "video-input"
output_dir = "output"
features = []
labels = []
datos_imagen_recortada = []
directorio = {'Mascara': [], 'Keypoints': [], 'Angles': [], 'Etiqueta': []}
for carpetas in os.listdir(videos):
    if carpetas == "FERNANDO":
        ruta = os.path.join(videos, carpetas)
        for video_file in os.listdir(ruta):
            try:
                etiqueta = carpetas
                print(carpetas +" "+video_file)
                video_path = os.path.join(ruta, video_file)
                directorio['Mascara'].append(procesarVideo(video_path))
                directorio['Keypoints'].append(np.array(getKeypoints(etiqueta)))
                directorio['Angles'].append(np.array(getAngles(etiqueta)))
                directorio['Etiqueta'].append(etiqueta)
            except Exception as e:
                print(e)
                continue


FERNANDO FERNANDO_01.mp4
FERNANDO FERNANDO_02.mp4
FERNANDO FERNANDO_03.mp4
FERNANDO FERNANDO_04.mp4


In [57]:
print(len(directorio['Mascara']))
print(len(directorio['Keypoints']))
print(len(directorio['Angles']))
print(len(directorio['Etiqueta']))
test = np.array(directorio['Keypoints'])

4
4
4
4


In [61]:
df_mascara = pd.DataFrame(directorio)
df_mascara.to_pickle("df_mascara.pkl")